## Cleaning up Atlas data - UEdin ASK
**Function**      : Preprocess netCDF files and restructure the dataset<br>
**Description**   : In this notebook serves to clean up Atlas data which is given in netcdf format and aggregate the data into a single file.<br>
**Return Values   : .nc files**<br>
**Note**          : All the data is saved to netCDF4 format. Note that data from different models may vary concerning the resolution and coordinates.<br>

In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd

### Path
Specify the path to the dataset and the place to save the outputs. <br>

In [2]:
# please specify data path
datapath = './AtlasData/raw'

# please specify output path
output_path = './AtlasData/preprocess'
os.makedirs(output_path, exist_ok = True)

Components used to create the output file names. Here, only `institution_id` and `cmor_var` is based on on CMIP DRS conventions.

In [5]:
output_file_name = {
    "prefix": "atlas",
    "activity": "EUCP", # project name e.g. EUCP
    "institution_id": "UEdin", # UEdin
    "source": "CMIP6", # e.g. CMIP6 or CMIP5
    "method": "ASK", # e.g. ASK
    "sub_method": "cons", # e.g. cons or uncons
    "cmor_var": "tas", # e.g. tas or pr
}

### Load raw data
Extract weather/climate data from given netCDF files.

In [6]:
# UEdin ASK
dataset_tas_djf = xr.open_dataset(os.path.join(datapath,'UEdin_ASK',
                          'UEdin_atlas_tas_CMIP6_histssp585_DJF_latlon.nc'))
dataset_tas_djf

<xarray.Dataset>
Dimensions:     (lon: 26, lat: 20)
Coordinates:
  * lon         (lon) float64 -16.25 -13.75 -11.25 -8.75 ... 41.25 43.75 46.25
  * lat         (lat) float64 28.75 31.25 33.75 36.25 ... 71.25 73.75 76.25
Data variables:
    avg_uncons  (lat, lon) float64 ...
    q10_uncons  (lat, lon) float64 ...
    q25_uncons  (lat, lon) float64 ...
    q50_uncons  (lat, lon) float64 ...
    q75_uncons  (lat, lon) float64 ...
    q90_uncons  (lat, lon) float64 ...
    avg_cons    (lat, lon) float64 ...
    q10_cons    (lat, lon) float64 ...
    q25_cons    (lat, lon) float64 ...
    q50_cons    (lat, lon) float64 ...
    q75_cons    (lat, lon) float64 ...
    q90_cons    (lat, lon) float64 ...
Attributes:
    creation_date:           27-Sep-2021 17:09:13
    projection_description:  CMIP6 projection of 2041-2060 change in Winter (...
    institution:             University of Edinburgh
    contact_email:           Andrew.Ballinger@ed.ac.uk

In [7]:
# CNRM/S HistC JJA
dataset_tas_jja = xr.open_dataset(os.path.join(datapath,'UEdin_ASK',
                                  'UEdin_atlas_tas_CMIP6_histssp585_JJA_latlon.nc'))
dataset_tas_jja

<xarray.Dataset>
Dimensions:     (lon: 26, lat: 20)
Coordinates:
  * lon         (lon) float64 -16.25 -13.75 -11.25 -8.75 ... 41.25 43.75 46.25
  * lat         (lat) float64 28.75 31.25 33.75 36.25 ... 71.25 73.75 76.25
Data variables:
    avg_uncons  (lat, lon) float64 ...
    q10_uncons  (lat, lon) float64 ...
    q25_uncons  (lat, lon) float64 ...
    q50_uncons  (lat, lon) float64 ...
    q75_uncons  (lat, lon) float64 ...
    q90_uncons  (lat, lon) float64 ...
    avg_cons    (lat, lon) float64 ...
    q10_cons    (lat, lon) float64 ...
    q25_cons    (lat, lon) float64 ...
    q50_cons    (lat, lon) float64 ...
    q75_cons    (lat, lon) float64 ...
    q90_cons    (lat, lon) float64 ...
Attributes:
    creation_date:           27-Sep-2021 17:08:38
    projection_description:  CMIP6 projection of 2041-2060 change in Summer (...
    institution:             University of Edinburgh
    contact_email:           Andrew.Ballinger@ed.ac.uk

### Preprocess data

Select Europe region

In [10]:
EU_LAT_BNDS = slice(30, 77)
EU_LON_BNDS = slice(-9, 39)

eu_dataset_tas_djf = dataset_tas_djf.sel(lon=EU_LON_BNDS, lat=EU_LAT_BNDS)
eu_dataset_tas_jja = dataset_tas_jja.sel(lon=EU_LON_BNDS, lat=EU_LAT_BNDS)

Create an empty xarray to host the processed data

In [11]:
VAR_NAME = "tas"
TIMES =  {"JJA": "2050-7-16", "DJF": "2051-1-16"} # "0000-4-16", "0000-7-16", "0000-10-16", "0000-1-16" MAM JJA SON DJF
PERCENTILES = [10, 25, 50, 75, 90]

LAT = eu_dataset_tas_djf.coords['lat']
LON = eu_dataset_tas_djf.coords['lon']

ds = xr.Dataset(
                {VAR_NAME: (("time", "latitude", "longitude","percentile"),
                 np.full([len(TIMES), len(LAT), len(LON), len(PERCENTILES)], np.nan)),
                "climatology_bounds": (pd.to_datetime(['2050-6-1', '2050-9-1', '2050-12-1', '2051-3-1']))},
                coords={    
                        "time": pd.to_datetime(list(TIMES.values())),
                        "latitude": LAT.values,
                        "longitude": LON.values, 
                        "percentile": PERCENTILES,
                 },
                 attrs={"description": "Contains modified UEdin ASK data used for Atlas in EUCP project.",
                       "history": "original UEdin ASK data files UEdin_atlas_tas_CMIP6_histssp585_DJF_latlon.nc, UEdin_atlas_tas_CMIP6_histssp585_JJA_latlon.nc" }
)

Fix attributes of each variable. Here, we follow CF-conventions as much as possible.

In [12]:
attrs = {"tas" : {"description":"Change in Air Temperature",
            "standard_name":"Change in Air Temperature",
            "long_name": "Change in Near-Surface Air Temperature",
            "units": "K", # in line with raw data
            "cell_methods":"time: mean changes over 20 years 2041-2060 vs 1995-2014",
           },
         "latitude": {"units": "degrees_north", "long_name": "latitude", "axis": "Y"},
         "longitude": {"units": "degrees_east", "long_name": "longitude", "axis": "X"},
         "time": {"climatology": "climatology_bounds",
                  "long_name": "time",
                  "axis": "T",
                  "climatology_bounds": ['2050-6-1', '2050-9-1', '2050-12-1', '2051-3-1'],
                  "description": "mean changes over 20 years 2041-2060 vs 1995-2014. The mid point 2050 is chosen as the representative time."},
         "percentile": {"units": "%", "long_name": "percentile", "axis": "Z"},
}
                
for key in attrs.keys():
    ds[key].attrs = attrs[key]
    
ds

<xarray.Dataset>
Dimensions:             (time: 2, latitude: 19, longitude: 20, percentile: 5, climatology_bounds: 4)
Coordinates:
  * climatology_bounds  (climatology_bounds) datetime64[ns] 2050-06-01 ... 20...
  * time                (time) datetime64[ns] 2050-07-16 2051-01-16
  * latitude            (latitude) float64 31.25 33.75 36.25 ... 73.75 76.25
  * longitude           (longitude) float64 -8.75 -6.25 -3.75 ... 36.25 38.75
  * percentile          (percentile) int64 10 25 50 75 90
Data variables:
    tas                 (time, latitude, longitude, percentile) float64 nan ....
Attributes:
    description:  Contains modified UEdin ASK data used for Atlas in EUCP pro...
    history:      original UEdin ASK data files UEdin_atlas_tas_CMIP6_histssp...

### Assemble data and save to netcdf

Make a function to assemble and save data 

In [13]:
def assembly(ds_original, ds_target, var, season, cons, percentile):
    """
    Select data from original nc files and save the target fields.
    """
    for j, percentile in enumerate(PERCENTILES):
        p = f"q{percentile}"
        if percentile == 50:
            p = "avg"
        ds_target[f"{var}"].values[list(TIMES).index(season),:,:,j] = ds_original[f"{p}_{cons}"]

Call the function

In [14]:
output_file_name["cmor_var"] = VAR_NAME 
for projection in ["uncons","cons"]:
    output_file_name["sub_method"] = projection
    
    # DJF
    assembly(eu_dataset_tas_djf, ds, "tas", "DJF", projection, ds.coords["percentile"].values[:])

    # JJA
    assembly(eu_dataset_tas_jja, ds, "tas", "JJA", projection, ds.coords["percentile"].values[:])
    
    file_name = f"{'_'.join(output_file_name.values())}.nc"
    print(f"one dataset is saved to {file_name}")
    ds.to_netcdf(os.path.join(output_path, file_name))

one dataset is saved to atlas_EUCP_UEdin_CMIP6_ASK_uncons_tas.nc
one dataset is saved to atlas_EUCP_UEdin_CMIP6_ASK_cons_tas.nc


### Check output

Load one of the saved data.

In [20]:
ds = xr.open_dataset(os.path.join(output_path,"atlas_EUCP_UEdin_CMIP6_ASK_cons_tas.nc"))
ds

<xarray.Dataset>
Dimensions:             (time: 2, latitude: 19, longitude: 20, percentile: 5, climatology_bounds: 4)
Coordinates:
  * climatology_bounds  (climatology_bounds) datetime64[ns] 2050-06-01 ... 20...
  * time                (time) datetime64[ns] 2050-07-16 2051-01-16
  * latitude            (latitude) float64 31.25 33.75 36.25 ... 73.75 76.25
  * longitude           (longitude) float64 -8.75 -6.25 -3.75 ... 36.25 38.75
  * percentile          (percentile) int64 10 25 50 75 90
Data variables:
    tas                 (time, latitude, longitude, percentile) float64 ...
Attributes:
    description:  Contains modified UEdin ASK data used for Atlas in EUCP pro...
    history:      original UEdin ASK data files UEdin_atlas_tas_CMIP6_histssp...